In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import syllables
import re
import stanza
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [4]:
#stanza.download('en', logging_level='WARN')
nlp = stanza.Pipeline('en', logging_level='WARN')

In [24]:
dfTrain = pd.read_csv('../Data/train.csv', encoding = 'utf-8', lineterminator="\n")
target = dfTrain['target'].to_frame()
target.columns = ['Values']

In [6]:
dfPOS = pd.read_csv('../Data/POS_Columns.csv', encoding = 'utf-8', lineterminator="\n")
cols = ['Index', 'Sents', 'Words', 'C_W_C', 'S_C',
                'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET','INTJ', 'NOUN', 
                'NUM', 'PART', 'PRON','PROPN', 'PUNCT','SCONJ','SYM','VERB','X']
dfPOS.columns = cols
dfPOS = dfPOS.drop(['Index'], axis = 1)
dfPOS = dfPOS.drop(['X'], axis = 1)

In [7]:
# print(len(doc.sentences))
# words = 0
# punct = 0
# for sentence in doc.sentences:
#     pos = []
# #     print(sentence)
#     words += len(sentence.words)
#     for word in sentence.words:
#         if word.pos == 'PUNCT':
#             punct += 1
#         pos.append(word.pos)
#         print(word)
#     print(pos)
# print(words - punct)

In [8]:
# X_array = []
# def convert_string(excerpt):
#     string = ''

#     for i in excerpt:
#         i = i.replace("\n", "")
#         string += i
#     return string

# for i in X:
#     doc = nlp(convert_string(i))
#     sentences = len(doc.sentences)
#     words = 0
#     syllable_count = 0
#     complex_word_count = 0
#     PartOfSpeech = {i:0 for i in PartOfSpeechTags}
#     for sentence in doc.sentences:
#         words += len(sentence.words)
#         for word in sentence.words:
#             PartOfSpeech[word.pos] += 1
#             if word.pos != 'PUNCT':
#                 syllable_count += syllables.estimate(word.text)
#                 if syllables.estimate(word.lemma) >= 3:
#                     complex_word_count += 1
#     words = words - PartOfSpeech['PUNCT']
#     X_array.append([sentences, words, complex_word_count, syllable_count, *PartOfSpeech.values()])

In [9]:
# X_array

In [10]:
# # X_array = np.array(X_array)
# pd.DataFrame(X_array).to_csv('../Data/POS.csv')

In [11]:
reg = LinearRegression()

In [12]:
lin_scores = cross_val_score(reg, dfPOS, targetValues, scoring="neg_mean_absolute_error", cv=10)

In [13]:
lin_scores = - lin_scores

In [14]:
lin_scores, lin_scores.mean(), lin_scores.std()

(array([0.58748557, 0.71196154, 0.7370633 , 0.71674263, 0.62605086,
        0.65625646, 0.68822282, 0.65913333, 0.5167674 , 0.65089023]),
 0.6550574135380979,
 0.06279724083745164)

In [15]:
tree_reg = DecisionTreeRegressor()
scores = cross_val_score(tree_reg, dfPOS,targetValues, scoring="neg_mean_absolute_error", cv=10)
scores = - scores
scores, scores.mean(), scores.std()

(array([0.88822063, 0.91902241, 1.00484853, 0.90930802, 0.90282073,
        0.90992352, 0.9300873 , 0.89107454, 0.79823857, 0.88698996]),
 0.9040534200271748,
 0.04799182098221051)

In [25]:
sdg_reg = SGDRegressor()
scaler = StandardScaler()
scaledMetric = scaler.fit_transform(np.array(dfPOS))
sdg_reg.fit(scaledMetric, target["Values"].values)

SGDRegressor()

In [27]:
sdg_scores = cross_val_score(sdg_reg, scaledMetric, target["Values"].values, scoring="neg_mean_absolute_error", cv=10)
sdg_scores = - sdg_scores
sdg_scores, sdg_scores.mean(), sdg_scores.std()

(array([0.58078475, 0.70931378, 0.74131034, 0.72724946, 0.62693891,
        0.65613795, 0.70038847, 0.65969366, 0.52062416, 0.65435869]),
 0.6576800187294538,
 0.06479480259446461)

In [28]:
gb_reg = GradientBoostingRegressor()
gb_reg.fit(scaledMetric, targetValues)
gb_scores = cross_val_score(gb_reg, scaledMetric, target["Values"].values, scoring="neg_mean_absolute_error", cv=10)
gb_scores = - gb_scores
gb_scores, gb_scores.mean(), gb_scores.std()

c:\users\raffi\documents\artificial_intelligence\artificial_intelligence\venv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


(array([0.60565252, 0.69112228, 0.74577564, 0.67986671, 0.6290814 ,
        0.63478085, 0.66404986, 0.66024507, 0.53402232, 0.65427444]),
 0.6498871081448964,
 0.053069078020693466)

In [29]:
rf_reg = RandomForestRegressor()
rf_reg.fit(scaledMetric, targetValues)
rf_scores = cross_val_score(rf_reg, scaledMetric, target["Values"].values, scoring="neg_mean_absolute_error", cv=10)
rf_scores = - rf_scores
rf_scores, rf_scores.mean(), rf_scores.std()

C:\Users\Raffi\AppData\Local\Temp/ipykernel_2912/3409595605.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_reg.fit(scaledMetric, targetValues)


(array([0.59430446, 0.68301034, 0.75745108, 0.69180459, 0.6515081 ,
        0.6388529 , 0.64387908, 0.66786243, 0.53171851, 0.66102186]),
 0.6521413337381655,
 0.05659879620916431)

In [31]:
# Create a pipeline that combines the preprocessor with linear regression
gbs = Pipeline([("scaler", StandardScaler()),
    ("predictor", GradientBoostingRegressor())])

# Create a dictionary of hyperparameters
gbs_param_grid = {"predictor__n_estimators": [100, 200, 300]}

# Create the grid search object which will find the best hyperparameter values based on validation error
gbs_gs = GridSearchCV(gbs, gbs_param_grid, scoring="neg_mean_absolute_error", cv=10)

# Run grid search by calling fit
gbs_gs.fit(dfPOS, target["Values"].values)

# Let's see how well we did
gbs_gs.best_params_, gbs_gs.best_score_

({'predictor__n_estimators': 300}, -0.6493952936725401)